In [1]:
import csv,sys
from collections import namedtuple
import numpy as np
import os
from PIL import Image
from sklearn.feature_extraction import image
from sklearn.feature_extraction.image import extract_patches
#from blockwise_view import blockwise_view
from skimage.util.shape import view_as_windows
import csv
from collections import namedtuple
import random

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 100000)
pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=sys.maxsize)

In [3]:
def blockwise_view(a, blockshape, aslist=False, require_aligned_blocks=True):
    """
    Return a 2N-D view of the given N-D array, rearranged so each ND block (tile)
    of the original array is indexed by its block address using the first N
    indexes of the output array.

    Note: This function is nearly identical to ``skimage.util.view_as_blocks()``, except:
          - "imperfect" block shapes are permitted (via require_aligned_blocks=False)
          - only contiguous arrays are accepted.  (This function will NOT silently copy your array.)
            As a result, the return value is *always* a view of the input.

    Args:
        a: The ND array

        blockshape: The tile shape

        aslist: If True, return all blocks as a list of ND blocks
                instead of a 2D array indexed by ND block coordinate.

        require_aligned_blocks: If True, check to make sure no data is "left over"
                                in each row/column/etc. of the output view.
                                That is, the blockshape must divide evenly into the full array shape.
                                If False, "leftover" items that cannot be made into complete blocks
                                will be discarded from the output view.

    Here's a 2D example (this function also works for ND):

    >>> a = np.arange(1,21).reshape(4,5)
    >>> print a
    [[ 1  2  3  4  5]
     [ 6  7  8  9 10]
     [11 12 13 14 15]
     [16 17 18 19 20]]

    >>> view = blockwise_view(a, (2,2), False)
    >>> print view
    [[[[ 1  2]
       [ 6  7]]
    <BLANKLINE>
      [[ 3  4]
       [ 8  9]]]
    <BLANKLINE>
    <BLANKLINE>
     [[[11 12]
       [16 17]]
    <BLANKLINE>
      [[13 14]
       [18 19]]]]

    Inspired by the 2D example shown here: http://stackoverflow.com/a/8070716/162094
    """
    assert a.flags['C_CONTIGUOUS'], "This function relies on the memory layout of the array."
    blockshape = tuple(blockshape)
    outershape = tuple(np.array(a.shape) // blockshape)
    view_shape = outershape + blockshape

    if require_aligned_blocks:
        assert (np.mod(a.shape, blockshape) == 0).all(), \
            "blockshape {} must divide evenly into array shape {}" \
                .format(blockshape, a.shape)

    # inner strides: strides within each block (same as original array)
    intra_block_strides = a.strides

    # outer strides: strides from one block to another
    inter_block_strides = tuple(a.strides * np.array(blockshape))

    # This is where the magic happens.
    # Generate a view with our new strides (outer+inner).
    view = np.lib.stride_tricks.as_strided(a,
                                              shape=view_shape,
                                              strides=(inter_block_strides + intra_block_strides))

    return view

In [4]:
result = []
masks = []
img = []
idx = []
hw = []

original_images = []
headers = []

In [5]:
patch_size = 64;
data = 'columbia'+str(patch_size)
base_folder="G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data"

In [6]:
original_images = [os.path.join(base_folder, '4cam_splc')+'/'+x for x in os.listdir(
    os.path.join(base_folder, '4cam_splc')) if x.endswith(".tif")]
print(len(original_images))

180


In [7]:
train_rows = []
val_rows = []
test_rows = []

for v in original_images:
    rr = random.random()
    if rr<= 0.65:
        train_rows.append(v)
    elif rr <=0.75:
        val_rows.append(v)
    else:
        test_rows.append(v)
        
print(len(train_rows))
print(len(val_rows))
print(len(test_rows))

119
15
46


In [8]:
dic = {
    'train':train_rows,
    'val':val_rows,
    'test':test_rows
}

In [9]:
for str_type in ['train','val','test']:

    count = 0
    lst = []
    
    # mkdirs
    os.makedirs(os.path.join(base_folder,data,'images',str_type))
    os.makedirs(os.path.join(base_folder,data,'masks',str_type)) 

    print("str_type ",str_type)
    for img_path in dic[str_type]: # each image in dic

        IMG_PATH = img_path
        MASK_PATH = img_path.replace(
            '4cam_splc', 'edgemask').replace('.tif', '_edgemask.jpg')
        print("IMG_PATH: ",IMG_PATH)
        print("MASK_PATH: ",MASK_PATH)

        prob = Image.open(IMG_PATH)
        rgbmask = Image.open(MASK_PATH) # here the mask is in rgb form, we need to change the non-spliced region to 255
        
        assert prob.size == rgbmask.size
        width,height = prob.size

        rgbmask = np.array(rgbmask)
        prob = np.array(prob)
        
        idx = np.argmax(rgbmask,2)

        mask = np.zeros([rgbmask.shape[0],rgbmask.shape[1]])
        mask[idx == 0] = 255 # rgb

        img_patches = blockwise_view(prob,(patch_size,patch_size,3),require_aligned_blocks=False)
        mask_patches = blockwise_view(mask,(patch_size,patch_size),require_aligned_blocks=False)

        img_patches = img_patches.squeeze(axis=2)


        for h in range(img_patches.shape[0]):
            for w in range(img_patches.shape[1]):
               
                im = Image.fromarray(img_patches[h][w])
                mk = Image.fromarray(mask_patches[h][w]).convert('RGB')

                PATCH_PATH = os.path.join(
                    data, 'images', str_type, str(count)+'.png')
                MASK_PATCH_PATH = os.path.join(
                    data, 'masks', str_type, str(count)+'.png')

                im.save(base_folder+'/'+PATCH_PATH)
                mk.save(base_folder+'/'+MASK_PATCH_PATH)

                lst.append(",".join([PATCH_PATH, MASK_PATCH_PATH, MASK_PATH.replace(
                    base_folder, ''), IMG_PATH.replace(base_folder, '')]))

                count = count + 1

        print('['+str(count)+']') 

    with open(os.path.join(base_folder,data,str_type+'.txt'),'w') as f:
        f.write("\n".join(lst))    

str_type  train
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_canonxt_sub_02.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_canonxt_sub_02_edgemask.jpg
[88]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_canonxt_sub_06.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_canonxt_sub_06_edgemask.jpg
[176]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_canonxt_sub_07.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_canonxt_sub_07_edgemask.jpg
[264]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulatio

[2376]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_kodakdcs330_sub_10.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_kodakdcs330_sub_10_edgemask.jpg
[2464]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_kodakdcs330_sub_12.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_kodakdcs330_sub_12_edgemask.jpg
[2552]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_kodakdcs330_sub_13.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_kodakdcs330_sub_13_edgemask.jpg
[2640]
IMG_PATH:  G:/sourav/prog/noteboo

[4664]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_nikond70_sub_23.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_nikond70_sub_23_edgemask.jpg
[4752]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_nikond70_sub_24.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_nikond70_sub_24_edgemask.jpg
[4840]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canong3_nikond70_sub_25.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canong3_nikond70_sub_25_edgemask.jpg
[4928]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulati

[9512]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canonxt_kodakdcs330_sub_26.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canonxt_kodakdcs330_sub_26_edgemask.jpg
[9728]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canonxt_kodakdcs330_sub_29.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canonxt_kodakdcs330_sub_29_edgemask.jpg
[9944]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canonxt_kodakdcs330_sub_30.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canonxt_kodakdcs330_sub_30_edgemask.jpg
[10160]
IMG_PATH:  G:/sourav/prog/notebo

[13610]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/nikond70_kodakdcs330_sub_10.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/nikond70_kodakdcs330_sub_10_edgemask.jpg
[13760]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/nikond70_kodakdcs330_sub_11.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/nikond70_kodakdcs330_sub_11_edgemask.jpg
[13910]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/nikond70_kodakdcs330_sub_13.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/nikond70_kodakdcs330_sub_13_edgemask.jpg
[14060]
IMG_PATH:  G:/sourav/pr

[1370]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/nikond70_canonxt_sub_28.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/nikond70_canonxt_sub_28_edgemask.jpg
[1520]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/nikond70_kodakdcs330_sub_04.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/nikond70_kodakdcs330_sub_04_edgemask.jpg
[1670]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/nikond70_kodakdcs330_sub_09.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/nikond70_kodakdcs330_sub_09_edgemask.jpg
[1820]
str_type  test
IMG_PATH:  G:/sourav

[2024]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canonxt_kodakdcs330_sub_08.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canonxt_kodakdcs330_sub_08_edgemask.jpg
[2240]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canonxt_kodakdcs330_sub_14.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canonxt_kodakdcs330_sub_14_edgemask.jpg
[2456]
IMG_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\4cam_splc/canonxt_kodakdcs330_sub_17.tif
MASK_PATH:  G:/sourav/prog/notebook/Image_manipulation_detection-master/image-splicing-localization-master/data\edgemask/canonxt_kodakdcs330_sub_17_edgemask.jpg
[2672]
IMG_PATH:  G:/sourav/prog/noteboo